In [1]:
import pandas as pd
import numpy as np

import pickle

In [2]:
col_interest = ["ID", "ageattest", "Gender", "Country", 
               "weight", "height", "bmi", "resting_sbp",
                "resting_hr", "waist", "bodyfat", "PhysInactive",
                "ANYCVD", "Diabetes", "Asthma", "COPD", "BetaMed",
                "Mode", "vo2_ml_kg_min", "max_rer", "peak_rpe"]

df = pd.read_excel("Data/FRIEND_dataset.xlsx", usecols=col_interest)

## Filter data to what we want.

In [3]:
# Keep only M/F.

df.query("Gender == 'Male' | Gender == 'Female'", inplace=True)
# df.value_counts("Gender")

In [4]:
# Keep only treadmill or cycling exercise test modes.
# (ref standards are based on these two modes only)

df.query("Mode == 'TM' | Mode == 'CY'", inplace=True)
# df.value_counts("Mode")

In [5]:
# Require RER ≥ 1.0 to indicate a maximal effort during the exercise test.

df.query("max_rer >= 1.0", inplace=True)

In [6]:
# Keeping the age range somewhat reasonable.

df.query("ageattest >= 20 & ageattest < 90", inplace=True)

## Clean data.

In [7]:
# Fix some data entry issues (prior to numeric conversion).

df["PhysInactive"] = np.where(df["PhysInactive"] == "1 (normally 0)", 1, df["PhysInactive"])
df["PhysInactive"] = np.where(df["PhysInactive"] == "na", np.nan, df["PhysInactive"])

df["Diabetes"] = np.where(df["Diabetes"] == "1 - type II", 1, df["Diabetes"])
df["Diabetes"] = np.where(df["Diabetes"] == "1 (Type 1)", 1, df["Diabetes"])

df["COPD"] = np.where(df["COPD"] == "na", np.nan, df["COPD"])

df["Asthma"] = np.where(df["Asthma"] == "na", np.nan, df["COPD"])


In [8]:
# Make a function to convert elements to numeric.

def make_num_exception(val):
    """
    Convert elements to numeric.
    If NOT able to convert, return None but print the element value.
    Can then go back and change as needed.
    """
    try:
        return float(val)
    except ValueError:
        print("Error found with entry: " + val)
        return None

In [9]:
# Convert COPD to numeric

df["COPD"] = df["COPD"].apply(make_num_exception)

In [10]:
# Convert PhysInactive to numeric

df["PhysInactive"] = df["PhysInactive"].apply(make_num_exception)

In [11]:
# Convert sBP to numeric

df["resting_sbp"] = df["resting_sbp"].apply(make_num_exception)

In [12]:
# Convert HR to numeric

df["resting_hr"] = df["resting_hr"].apply(make_num_exception)

Error found with entry: N/a
Error found with entry: N/a


In [13]:
# Convert Diabetes to numeric

df["Diabetes"] = df["Diabetes"].apply(make_num_exception)

In [14]:
# Convert Asthma to numeric

df["Asthma"] = df["Asthma"].apply(make_num_exception)

In [15]:
# Convert those with zeroes in certain columns to missing (and print how many 0's there were).

no_zeroes = ["ageattest", "height", "weight", "vo2_ml_kg_min", "max_rer"]

for x in no_zeroes:
    temp_ = df[x] == 0
    print("There were: " + str(temp_.sum()) + " 0's in " + x)    
    df[x] = np.where(df[x] == 0, np.nan, df[x])

There were: 0 0's in ageattest
There were: 0 0's in height
There were: 0 0's in weight
There were: 0 0's in vo2_ml_kg_min
There were: 0 0's in max_rer


In [16]:
# Combine some data groups.

# For this I don't care if type 1 or 2.
df["Diabetes"] = np.where(df["Diabetes"] > 1, 1, df["Diabetes"])

df["COPD"] = np.where(df["COPD"] > 1, 1, df["COPD"])

df["BetaMed"] = np.where(df["BetaMed"] > 1, 1, df["BetaMed"])

df["ANYCVD"] = np.where(df["ANYCVD"] > 1, 1, df["ANYCVD"])

In [17]:
# Convert unrealistic values to missing.

df["waist"] = np.where(df["waist"] > 200, np.nan, df["waist"])
df["waist"] = np.where(df["waist"] < 40, np.nan, df["waist"])

df["bodyfat"] = np.where(df["bodyfat"] > 75, np.nan, df["bodyfat"])

df["resting_hr"] = np.where(df["resting_hr"] > 200, np.nan, df["resting_hr"])
df["resting_hr"] = np.where(df["resting_hr"] < 20, np.nan, df["resting_hr"])

df["resting_sbp"] = np.where(df["resting_sbp"] < 50, np.nan, df["resting_sbp"])

df["vo2_ml_kg_min"] = np.where(df["vo2_ml_kg_min"] > 99, np.nan, df["vo2_ml_kg_min"])
df["vo2_ml_kg_min"] = np.where(df["vo2_ml_kg_min"] < 1, np.nan, df["vo2_ml_kg_min"])

## Feature engineering.

In [18]:
# Create the fitness categories (the target).
# "Low" is defined as <20 ml/kg/min, which is roughly the fitness needed to do 1.5 stages of the Bruce protocol.

df["fitness_category"] = df["vo2_ml_kg_min"].apply(lambda x: "Low" if x < 20 else "OK")

df.fitness_category.value_counts()

OK     49332
Low    29013
Name: fitness_category, dtype: int64

In [19]:
# Create BMI categories.
# Combined underweight and healthy bmi due to low number of underweight (~600).

bmi_bins = [0, 25, 30, 100]
bmi_labels = ["under/healthy", "overweight", "obese"]

df["bmi_cat"] = pd.cut(df["bmi"], bins=bmi_bins, labels=bmi_labels, right=False)

df.bmi_cat.value_counts()

overweight       30145
under/healthy    24836
obese            22922
Name: bmi_cat, dtype: int64

In [20]:
# Create a hypertension category.
# (for this project, just going to use sBP and ignore dBP cut-points)

df["hypertension"] = df["resting_sbp"].apply(lambda x: 0 if x < 140 else 1)

df.hypertension.value_counts()

0    49128
1    29217
Name: hypertension, dtype: int64

## Final filter to include those without missing values (for main variables of interest).

In [21]:
# Requiring these columns to all have values present.

check_nan = ["ageattest", "height", "weight", "vo2_ml_kg_min", "bmi", "PhysInactive",
            "resting_hr", "resting_sbp", "ANYCVD", "Diabetes", "Asthma", "COPD", "BetaMed"]

In [22]:
df.dropna(axis="rows", how="any", subset=check_nan, inplace=True)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6252 entries, 3196 to 83156
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   ID                6225 non-null   object  
 1   ageattest         6252 non-null   float64 
 2   Gender            6252 non-null   object  
 3   Country           6047 non-null   object  
 4   height            6252 non-null   float64 
 5   weight            6252 non-null   float64 
 6   waist             5608 non-null   float64 
 7   bodyfat           5872 non-null   float64 
 8   BetaMed           6252 non-null   float64 
 9   ANYCVD            6252 non-null   float64 
 10  Diabetes          6252 non-null   float64 
 11  Asthma            6252 non-null   float64 
 12  COPD              6252 non-null   float64 
 13  PhysInactive      6252 non-null   float64 
 14  Mode              6252 non-null   object  
 15  resting_hr        6252 non-null   float64 
 16  resting_sbp       62

In [24]:
with open('./data/cleaned_dataframe.pickle', 'wb') as to_write:
    pickle.dump(df, to_write)